In [50]:
# from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


mydict = {}

hunt_file = "Data\\Hunt\\J_A+A_673_A114_clusters.dat.gz.fits"
kharchenko_file = 'Data\\Kharchenko\\J_A+A_558_A53_catalog.dat.fits'
xmatch_file = 'Data\\Hunt\\J_A+A_673_A114_crossma.dat.fits'
dias_file = 'Data\\Dias\\B_ocl_clusters.dat.fits'

def datahandler(filename):
    authorname = filename.split('\\')
    # print(authorname)
    # mydict[authorname] = {}
    hdul = fits.open(filename)
    hdr1 = hdul[0].header #first and second header
    hdr2 = hdul[1].header
    data, cols = hdul[1].data, hdul[1].columns
    metadata = [ [ hdr2[f"TTYPE{i+1}"], hdr2.comments[f"TTYPE{i+1}"] ] for i, j in enumerate(cols.names) ]
    # mydict[authorname]['hdul'] = hdul
    return hdul[1].data, hdul[1].columns, metadata


data_hunt, cols_hunt, meta_hunt = datahandler(hunt_file)
# data_khar, cols_khar, meta_khar = datahandler(kharchenko_file)

## Comparing catalogues

The goal now is to get the correct clusters from the Kharchenko, Dias and CanatGaudin catalogues with respect to the Hunt paper. Pretty much recreate the table 4 from the hunt paper. We want to cross match the catalogues. We first want to find the $n_\mathrm{clusters}$ for each catalogue.

In [51]:
xmatch, xmatch_cols, meta_xmatch = datahandler(xmatch_file)
meta_xmatch

[['ID', '? Internal cluster ID (blank if object has no'],
 ['NameLit', 'Name in literature catalogue'],
 ['SourceCat', 'Source catalogue reference'],
 ['TypeSourceCat', 'Comma separated list of all source catalogue'],
 ['Sep', '[0/2]? Separation between cluster centres'],
 ['SepTidal', '[0/0.94]? Separation in terms of largest tidal'],
 ['SepTidalLit', '[0/1]? Separation in terms of literature tidal'],
 ['SepTidalData', '[0/1]? Separation in terms of tidal radius in'],
 ['pmRASep', '[0/17.41]? Separation in pmRA times cos(dec)'],
 ['pmRASigma', '? pmRA separation in terms of uncertainty on'],
 ['pmDESep', '[0/11.24]? Separation in pmDE between this work'],
 ['pmDESigma', '? pmDE separation in terms of uncertainty on'],
 ['PlxSep', '[0/2.8]? Separation in parallax between this'],
 ['PlxSigma', '? Parallax separation in terms of uncertainty'],
 ['maxSigma', '? Maximum value of pmRASepSigma, pmDESepSigma,'],
 ['meanSigma', '? Mean value of pmRASepSigma, pmDESepSigma, and']]

In [ ]:
hunt_file.split('\\')[1]

'Hunt'